<a href="https://colab.research.google.com/github/Annes0510/aaiffel_exp/blob/mian/8_Seq2seq%EC%9C%BC%EB%A1%9C_%EB%B2%88%EC%97%AD%EA%B8%B0_%EB%A7%8C%EB%93%A4%EA%B8%B0_Going_Deeper(NLP)_YJ2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager.findfont(font)

print("완료!")

8-2. 데이터 전처리

In [ ]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io
import pandas as pd

print(tf.__version__)

8-7. 프로젝트: 한영 번역기 만들기

In [ ]:
import pandas
import tensorflow
import matplotlib
import os

print(pandas.__version__)
print(tensorflow.__version__)
print(matplotlib.__version__)

Step 1. 데이터 다운로드

In [ ]:
import tensorflow as tf
import os

print(tf.__version__)

In [ ]:
path_to_zip_ko = tf.keras.utils.get_file('korean-english-park.train.tar.gz',
                                     origin = 'https://raw.githubusercontent.com/jungyeul/korean-parallel-corpora/master/korean-english-news-v1/korean-english-park.train.tar.gz',
                                     extract=True
                                     )

path_ko = os.path.dirname(path_to_zip_ko)+'/korean-english-park.train.ko'

In [ ]:
with open(path_ko, 'r') as f:
    raw_ko = f.read().splitlines()
    
print('Data Size:', len(raw_ko))
print('Example:')

for sen in raw_ko[0:100][::20]:
    print('>>', sen)


In [ ]:
path_to_zip_en = tf.keras.utils.get_file('korean-english-park.train.tar.gz',
                                     origin = 'https://raw.githubusercontent.com/jungyeul/korean-parallel-corpora/master/korean-english-news-v1/korean-english-park.train.tar.gz',
                                     extract=True
                                     )

path_en = os.path.dirname(path_to_zip_en)+'/korean-english-park.train.en'

In [ ]:
with open(path_en, 'r') as f:
    raw_en = f.read().splitlines()
    
print('Data Size:', len(raw_en))
print('Example:')

for sen in raw_en[0:100][::20]:
    print('>>', sen)

Step 2. 데이터 정제

In [ ]:
raw_ko_df = pd.DataFrame(raw_ko)
raw_en_df = pd.DataFrame(raw_en)

In [ ]:
df = pd.concat([raw_ko_df,raw_en_df],axis=1)
df.columns = ["ko","en"] 
df = df.drop_duplicates(["ko"], keep='first') #열기준으로 중복행 제거
df = df.drop_duplicates(["en"], keep='first') #열기준으로 중복행 제거
print('중복 제거 후 데이터 수: ',len(df))

In [ ]:
df.head()

In [ ]:
raw_ko = df["ko"].to_list()
raw_en = df["en"].to_list()

In [ ]:
# konlpy, Mecab 형태소 분석기 설치 스크립트 실행
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
# 정상 동작 확인
from konlpy.tag import Okt, Mecab

okt = Okt()
mecab = Mecab()


In [ ]:
#rawko 전처리 함수
def preprocess(sentences1,sentences2):
    enc_corpus_temp1 = []
    dec_corpus_temp1 = []
    enc_corpus_temp2 = []
    dec_corpus_temp2 = []
    enc_corpus = []
    dec_corpus = []
    for sentence in sentences1:
        #불필요한 이상한 기호가 존재, 특수문자는 불필요한 노이즈로 작용할 수 있기 때문에 삭제
        sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
        sentence = re.sub(r'[" "]+', " ", sentence)
        #한글에 적용할 수 있는 정규식 추가
        sentence = re.sub(r"[^ㄱ-ㅎ가-힣0-9?.!,]+", " ", sentence)
        #한글 토큰화는 KoNLPy의 mecab 클래스를 사용
        sentence = mecab.morphs(sentence)
        enc_corpus_temp1.append(sentence)
    
    for sentence in sentences2:
        sentence = sentence.lower().strip()

        #불필요한 이상한 기호가 존재, 특수문자는 불필요한 노이즈로 작용할 수 있기 때문에 삭제
        sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
        sentence = re.sub(r'[" "]+', " ", sentence)
        sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
        sentence = sentence.strip()
        sentence = '<start> ' + sentence
        sentence = sentence + ' <end>'
        sentence = sentence.strip()
        sentence = sentence.split()

        dec_corpus_temp1.append(sentence)
    
    for i in range(len(dec_corpus_temp1)):
        if len(dec_corpus_temp1[i])<=40:
            enc_corpus_temp2.append(enc_corpus_temp1[i])
            dec_corpus_temp2.append(dec_corpus_temp1[i])
            
    for i in range(len(enc_corpus_temp2)):
        if len(enc_corpus_temp2[i])<=40:
            enc_corpus.append(enc_corpus_temp2[i])
            dec_corpus.append(dec_corpus_temp2[i])
            
    return enc_corpus, dec_corpus

In [ ]:
enc_corpus, dec_corpus = preprocess(raw_ko, raw_en)

In [ ]:
enc_corpus[2]

In [ ]:
dec_corpus[0]

Step 3. 데이터 토큰화

In [ ]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', num_words=20000)
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, tokenizer

In [ ]:
# 토큰화하기
enc_tensor, enc_tokenizer = tokenize(enc_corpus)
dec_tensor, dec_tokenizer = tokenize(dec_corpus)
# 훈련 데이터와 검증 데이터로 분리하기
enc_train, enc_val, dec_train, dec_val = train_test_split(enc_tensor, dec_tensor, test_size=0.2)

print("Korean Vocab Size:", len(enc_tokenizer.index_word))
print("English Vocab Size:", len(dec_tokenizer.index_word))



Step 4. 모델 설계

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_dec = tf.keras.layers.Dense(units)
        self.w_enc = tf.keras.layers.Dense(units)
        self.w_com = tf.keras.layers.Dense(1)
    
    def call(self, h_enc, h_dec):
        # h_enc shape: [batch x length x units]
        # h_dec shape: [batch x units]

        h_enc = self.w_enc(h_enc)
        h_dec = tf.expand_dims(h_dec, 1)
        h_dec = self.w_dec(h_dec)

        score = self.w_com(tf.nn.tanh(h_dec + h_enc))
        
        attn = tf.nn.softmax(score, axis=1)

        context_vec = attn * h_enc
        context_vec = tf.reduce_sum(context_vec, axis=1)

        return context_vec, attn


In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(enc_units,
                                       return_sequences=True)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.gru(out)
        
        return out

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(dec_units,
                                       return_sequences=True,
                                       return_state=True)
        self.fc = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, h_dec, enc_out):
        context_vec, attn = self.attention(enc_out, h_dec)

        out = self.embedding(x)
        out = tf.concat([tf.expand_dims(context_vec, 1), out], axis=-1)
        
        out, h_dec = self.gru(out)
        out = tf.reshape(out, (-1, out.shape[2]))
        out = self.fc(out)

        return out, h_dec, attn

In [ ]:
BATCH_SIZE     = 64
SRC_VOCAB_SIZE = len(enc_tokenizer.index_word) + 1
TGT_VOCAB_SIZE = len(dec_tokenizer.index_word) + 1

units         = 1024
embedding_dim = 512

encoder = Encoder(SRC_VOCAB_SIZE, embedding_dim, units)
decoder = Decoder(TGT_VOCAB_SIZE, embedding_dim, units)

# sample input
sequence_len = 30

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_enc)

print ('Encoder Output:', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))

sample_logits, h_dec, attn = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                     sample_state, sample_output)

print ('Decoder Output:', sample_logits.shape)
print ('Decoder Hidden State:', h_dec.shape)
print ('Attention:', attn.shape)

Step 5. 훈련하기

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    
    return tf.reduce_mean(loss)

In [ ]:
@tf.function
def train_step(src, tgt, encoder, decoder, optimizer, dec_tok):
    bsz = src.shape[0]
    loss = 0

    with tf.GradientTape() as tape:
        enc_out = encoder(src)
        h_dec = enc_out[:, -1]
        
        dec_src = tf.expand_dims([dec_tok.word_index['<start>']] * bsz, 1)

        for t in range(1, tgt.shape[1]):
            pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

            loss += loss_function(tgt[:, t], pred)
            dec_src = tf.expand_dims(tgt[:, t], 1)
        
    batch_loss = (loss / int(tgt.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss

In [ ]:
from tqdm import tqdm

EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss = train_step(enc_train[idx:idx+BATCH_SIZE],
                                dec_train[idx:idx+BATCH_SIZE],
                                encoder,
                                decoder,
                                optimizer,
                                dec_tokenizer)
    
        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))
    
    test_loss = 0
    
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (test_batch, idx) in enumerate(t):
        test_batch_loss = train_step(enc_train[idx:idx+BATCH_SIZE],
                                    dec_train[idx:idx+BATCH_SIZE],
                                    encoder,
                                    decoder,
                                    optimizer, 
                                    dec_tokenizer)
    
        test_loss += batch_loss

        t.set_description_str('Test Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Test Loss %.4f' % (test_loss.numpy() / (test_batch + 1)))


In [ ]:
def preprocess_sentence(sentence):

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)   
    sentence = re.sub(r"[^ㄱ-ㅎ가-힣0-9.,?!]+", " ", sentence)
    result = mecab.morphs(sentence)

    return result

In [ ]:
def evaluate(sentence, encoder, decoder):
    attention = np.zeros((dec_tensor.shape[-1], enc_tensor.shape[-1]))
    
    sentence = preprocess_sentence(sentence)
    inputs = enc_tokenizer.texts_to_sequences([sentence])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=enc_tensor.shape[-1], padding='post')

    result = ''

    enc_out = encoder(inputs)

    dec_hidden = enc_out[:, -1]
    dec_input = tf.expand_dims([dec_tokenizer.word_index['<start>']], 0)

    for t in range(dec_tensor.shape[-1]):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention[t] = attention_weights.numpy()

        predicted_id = tf.argmax(tf.math.softmax(predictions, axis=-1)[0]).numpy()

        result += dec_tokenizer.index_word[predicted_id] + ' '

        if dec_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence, attention

        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention


def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def translate(sentence, encoder, decoder):
    result, sentence, attention = evaluate(sentence, encoder, decoder)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    
    attention = attention[:len(result), :len(sentence)]
    plot_attention(attention, sentence, result.split(' '))

In [ ]:
import matplotlib.font_manager as fm
import matplotlib as mpl

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic')
mpl.font_manager._rebuild()

In [ ]:
translate("오바마는 대통령이다.", encoder, decoder)

In [ ]:
translate("시민들은 도시 속에 산다.", encoder, decoder)

In [ ]:
translate("커피는 필요 없다.", encoder, decoder)

In [ ]:
translate("일곱 명의 사망자가 발생했다.", encoder, decoder)

*회고록 

에포크를 돌리는데 하루 종일 걸렸다.  GPU로 변경해서 해보았지만 여전히 느려서 속이 터졌다. 

손실을 줄이기 위해 옵티마이저를 추가를 하였으나 번역 결과를 보니 비슷하나 정확도가 떨어졌다. 7명을 8명으로 번역이 되었다. RNN에서 시퀀스 투 시퀀스에 대해서 발표한 적이 있어서 구조에 대해서는 알고 있지만 이렇게 실습으로 연결되는 것 까지 이해하는데는 조금 어려웠다. 

노드를 참고해서 코드를 작성했는데 변수 메서드가 바뀌다 보니 오류가 생겨서 수정하다 보니 시각화가 찌부 되었다. 

노드까지 런타임을 다시 돌리면 제출이 불가할 것 같아서, 프로젝트만 돌리고 제출한다. 